In [ ]:
## 대대적인 개편 필요
## https://www.investopedia.com/terms/h/hhi.asp (HHI)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.preprocessing import StandardScaler # -1 ~ 1
from sklearn.preprocessing import MinMaxScaler # 0 ~ 1
path = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/real_code/pqt_file/'
plt.rc('axes', unicode_minus = False) # 마이너스 기호 깨질 때 사용 
plt.rc('font', family='Malgun Gothic') # 맑은 고딕 한글 사용

In [2]:
# 재무적인 전체 기업들 
tobinsQ_tt= pd.read_parquet(path + 'tobinsQ_final.pqt', engine = 'pyarrow') 
# 조달에 참여한 기업들(조달성공 + 조달실패)
bid_tt = pd.read_parquet(path + 'bid_nodemand.pqt', engine = 'pyarrow') # 수요기관 없는 데이터 사용
col = ['연도', '분기', '사업자등록번호']
total_df = pd.merge(tobinsQ_tt, bid_tt, how = 'left', on = col) # tobinsQ 데이터를 기준으로 left-join
total_df.drop(columns = ['symbol', '사업자등록번호'], axis = 1, inplace = True)
total_df.rename(columns={'entity_name': '회사명', 'market':'시장'}, inplace = True)
total_df.drop(columns = ['주가평균','도로명주소'], axis = 1, inplace = True) # 주가평균을 사용하지 않아 삭제 
total_df.fillna(0, inplace = True)
total_df['낙찰여부'] = (total_df['낙찰수']) > 0 
total_df['입찰여부'] = (total_df['입찰횟수']) > 0 
total_df['영업이익률'] = (total_df['영업이익'] / total_df['매출']) 
total_df.insert(5, '영업이익률', total_df.pop('영업이익률'))
total_df.drop(total_df[total_df['매출'] == 0].index, axis = 0, inplace = True)
total_df.reset_index(drop=True, inplace=True)
total_df

,회사명,연도,분기,매출,영업이익,영업이익률,총당기순이익,연구개발비,TobinsQ,업종분류코드,시장,종업원수,입찰횟수,낙찰사업평균참여수,낙찰금액,낙찰수,낙찰여부,입찰여부
0,3S,2020,1Q,2.259202e+10,4.671810e+08,0.020679,0.000000e+00,5.323600e+07,2.191006,C,KOSDAQ,75,1.0,4.0,4.800000e+08,1.0,True,True
1,3S,2020,2Q,4.265512e+09,-3.499970e+08,-0.082053,-4.673000e+08,8.397100e+07,2.289685,C,KOSDAQ,77,0.0,0.0,0.000000e+00,0.0,False,False
2,3S,2020,3Q,1.097065e+10,-8.039280e+08,-0.073280,-1.181574e+09,9.066300e+07,2.044231,C,KOSDAQ,77,0.0,0.0,0.000000e+00,0.0,False,False
3,3S,2020,4Q,1.973514e+10,-6.666700e+08,-0.033781,-9.868740e+08,9.793400e+07,1.862129,C,KOSDAQ,67,8.0,4.0,2.221000e+09,5.0,True,True
4,3S,2021,1Q,2.347791e+10,-1.408266e+09,-0.059983,6.953350e+08,9.362800e+07,1.974030,C,KOSDAQ,77,0.0,0.0,0.000000e+00,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23435,힘스,2021,4Q,5.058420e+10,-4.371187e+09,-0.086414,-4.413920e+08,3.632138e+09,0.967528,C,KOSDAQ,223,0.0,0.0,0.000000e+00,0.0,False,False
23436,힘스,2022,1Q,9.017422e+09,-2.198637e+09,-0.243821,-1.605159e+09,9.508470e+08,1.062566,C,KOSDAQ,227,0.0,0.0,0.000000e+00,0.0,False,False
23437,힘스,2022,2Q,1.822716e+10,-6.597300e+09,-0.361949,-4.122215e+09,1.963682e+09,0.880589,C,KOSDAQ,215,0.0,0.0,0.000000e+00,0.0,False,False
23438,힘스,2022,3Q,2.393546e+10,-1.253413e+10,-0.523663,-6.750329e+09,3.027635e+09,0.820003,C,KOSDAQ,215,0.0,0.0,0.000000e+00,0.0,False,False


In [3]:
total_df.describe() # 연구개발비 moving average 3개년치 평균 연구개발비 대비 그 시기 연구개발비 비용 // 3개년도 합계 0일 경우 다 날리자   

,연도,매출,영업이익,영업이익률,총당기순이익,연구개발비,TobinsQ,종업원수,입찰횟수,낙찰사업평균참여수,낙찰금액,낙찰수
count,23440.000000,2.344000e+04,2.344000e+04,23440.000000,2.344000e+04,2.344000e+04,23440.000000,2.344000e+04,23440.000000,23440.000000,2.344000e+04,23440.000000
mean,2021.025555,6.917971e+11,4.516792e+10,-1.227760,3.585275e+10,1.503053e+10,1.216811,4.919991e+04,0.595862,0.787815,2.072169e+08,0.103968
std,0.809825,5.322658e+12,7.547494e+11,36.503202,6.810462e+11,3.467024e+11,1.790592,5.364994e+06,6.047639,33.236027,4.515647e+09,0.835852
min,2020.000000,3.667000e+06,-3.265515e+13,-3746.544324,-2.442911e+13,0.000000e+00,0.000244,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,2020.000000,2.285277e+10,-9.265590e+08,-0.028727,-1.237357e+09,0.000000e+00,0.422355,8.900000e+01,0.000000,0.000000,0.000000e+00,0.000000
50%,2021.000000,6.629937e+10,2.414543e+09,0.035295,1.711654e+09,4.265880e+08,0.754423,1.890000e+02,0.000000,0.000000,0.000000e+00,0.000000
75%,2022.000000,2.220411e+11,1.291267e+10,0.092024,1.059608e+10,2.184132e+09,1.362350,4.290000e+02,0.000000,0.000000,0.000000e+00,0.000000
max,2022.000000,3.022314e+14,5.163386e+13,0.832342,5.565408e+13,2.500422e+13,42.231655,6.952446e+08,456.000000,4321.000000,3.643000e+11,62.000000


In [4]:
## IQR을 진행할경우 5000개가 drop되기 때문에 다른 방법 고사// 
# 영업이익률: 이상치 제거 (-4000 이거 어떻게 할지 고민)
# TobinsQ: log1p
# 연구개발비: 억단위로 짜른 이후 log transformation   
# 낙찰금액: 억단위로 짜른 이후 log transformation

In [5]:
# IQR을 사용한 이상치 제거 - 이건 아무리 봐도 아닌거 같다. 과연 높이 튀는 값들이 이상치들일까? 저 값들이 문제가 있다고 보기는 어렵다. 데이터가 얼마 없는 이상황에서 2천개, 5천개 데이터 삭제하는 방법은 옳지 않다고 생각한다. 
def get_outlier(df = None, col = None): 
    out = df[col]
    quantile_25 = np.percentile(out.values, 25)
    quantile_75 = np.percentile(out.values, 75) 
    iqr = quantile_75 - quantile_25
    low_val = quantile_25 - (1.5 * iqr)
    high_val = quantile_75 + (1.5 * iqr)
    out_index = out[(out < low_val) | (out > high_val)].index
    return out_index

# out_indx = get_outlier(total_df, 'TobinsQ') 
# total_df.drop(out_indx, axis = 0, inplace = True) # 토빈스 이상치 
out_indx1 = get_outlier(total_df, '영업이익률')
total_df.drop(out_indx1, axis = 0, inplace = True) # 영업이익률 이상치 약 3000개 삭제
total_df.reset_index(drop = True, inplace = True)

total_df.describe() # 20134개

,연도,매출,영업이익,영업이익률,총당기순이익,연구개발비,TobinsQ,종업원수,입찰횟수,낙찰사업평균참여수,낙찰금액,낙찰수
count,20134.000000,2.013400e+04,2.013400e+04,20134.000000,2.013400e+04,2.013400e+04,20134.000000,2.013400e+04,20134.000000,20134.000000,2.013400e+04,20134.000000
mean,2021.021854,7.790608e+11,5.340114e+10,0.043511,4.305797e+10,1.650407e+10,0.972133,9.693064e+02,0.659680,0.897235,2.343588e+08,0.115973
std,0.810341,5.686002e+12,7.424481e+11,0.087007,6.876366e+11,3.723564e+11,1.170730,9.513228e+03,6.429325,35.854261,4.822409e+09,0.896585
min,2020.000000,7.035430e+08,-5.846499e+12,-0.209388,-5.215581e+12,0.000000e+00,0.000244,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,2020.000000,3.143012e+10,8.938725e+07,0.002398,0.000000e+00,0.000000e+00,0.388978,1.030000e+02,0.000000,0.000000,0.000000e+00,0.000000
50%,2021.000000,8.290120e+10,3.206938e+09,0.041915,2.455074e+09,3.945830e+08,0.677731,2.130000e+02,0.000000,0.000000,0.000000e+00,0.000000
75%,2022.000000,2.641685e+11,1.448974e+10,0.092972,1.172053e+10,2.074700e+09,1.169020,4.710000e+02,0.000000,0.000000,0.000000e+00,0.000000
max,2022.000000,3.022314e+14,5.163386e+13,0.272725,5.565408e+13,2.500422e+13,42.231655,1.045154e+06,456.000000,4321.000000,3.643000e+11,62.000000


In [6]:
total_df.연구개발비.describe()

count    2.013400e+04
mean     1.650407e+10
std      3.723564e+11
min      0.000000e+00
25%      0.000000e+00
50%      3.945830e+08
75%      2.074700e+09
max      2.500422e+13
Name: 연구개발비, dtype: float64

In [7]:
total_df['연구개발비'] = total_df['연구개발비'].apply(lambda x: x/1e+8)
display(total_df['연구개발비'].describe(), np.log1p(total_df['연구개발비']).describe())
# 결과해석시 원래_값 = exp(변환된_값) - 1 으로 변환해석 해석

count     20134.000000
mean        165.040727
std        3723.564006
min           0.000000
25%           0.000000
50%           3.945830
75%          20.747003
max      250042.160000
Name: 연구개발비, dtype: float64

count    20134.000000
mean         1.852031
std          1.859313
min          0.000000
25%          0.000000
50%          1.598545
75%          3.079476
max         12.429389
Name: 연구개발비, dtype: float64

In [8]:
# total_df['낙찰금액'] = total_df['낙찰금액'].apply(lambda x: x/1e+8)
display(total_df['낙찰금액'].describe(), np.log1p(total_df['낙찰금액']).describe())
# 결과해석시 원래_값 = exp(변환된_값) - 1 으로 변환해석 해석

count    2.013400e+04
mean     2.343588e+08
std      4.822409e+09
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.643000e+11
Name: 낙찰금액, dtype: float64

count    20134.000000
mean         1.065392
std          4.460509
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         26.621244
Name: 낙찰금액, dtype: float64

In [9]:
## 여기서부터 시작하면 됨 

In [10]:
total_df.to_parquet('total_nodemand.pqt') # 최종 테이블 완성시 필요 데이터

## 수요기관 있는 메인 데이터 

In [47]:
bid_total = pd.read_parquet(path + 'bid_final.pqt', engine = 'pyarrow')
bid_total.head()

,연도,분기,사업자등록번호,낙찰금액,입찰횟수(공기업),평균참여수(공기업),낙찰금액(공기업),낙찰수(공기업),입찰횟수(지자체),평균참여수(지자체),낙찰금액(지자체),낙찰수(지자체),입찰횟수(국가기관),평균참여수(국가기관),낙찰금액(국가기관),낙찰수(국가기관)
0,2020,1Q,1010116082,0.0,0,0.0,0.0,0.0,11,0.0,0.0,0.0,1,0.0,0.0,0.0
1,2020,1Q,1010116587,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1,0.0,0.0,0.0
2,2020,1Q,1010118644,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,2020,1Q,1010145888,540790.0,0,0.0,0.0,0.0,7,4.5,540790.0,2.0,0,0.0,0.0,0.0
4,2020,1Q,1010146043,0.0,16,0.0,0.0,0.0,12,0.0,0.0,0.0,79,0.0,0.0,0.0


In [48]:
# ## 수요기관구분 있는 데이터 프레임
col = ['연도', '분기', '사업자등록번호']
total_df1 = pd.merge(tobinsQ_tt, bid_total, how = 'left', on = col)
total_df1.drop(columns = ['symbol', '사업자등록번호'], axis = 1, inplace = True)
total_df1.rename(columns={'entity_name': '회사명', 'market':'시장'}, inplace = True)
total_df1.drop(columns = ['주가평균','도로명주소'], axis = 1, inplace = True)
total_df1.fillna(0, inplace = True)
total_df1['낙찰여부'] = (total_df1['낙찰수(공기업)'] + total_df1['낙찰수(지자체)'] + total_df1['낙찰수(국가기관)']) > 0 
total_df1['입찰여부'] = (total_df1['입찰횟수(공기업)'] + total_df1['입찰횟수(지자체)'] + total_df1['입찰횟수(국가기관)']) > 0 
total_df1['시기'] =  total_df1['연도'].astype('str') + total_df1['분기'].astype('str').apply(lambda x : x[0]) # fixed effect 모델을 사용하기 위해 연도+분기 = 시기 정함 
total_df1['시기'] = total_df1['시기'].astype(int) 
total_df1.insert(1, '시기', total_df1.pop('시기')) # ['시기']열 데이터프레임 앞에 생성
total_df1.drop(columns = {'연도', '분기'}, inplace = True)
total_df1['영업이익률'] = (total_df1['영업이익'] / total_df1['매출']) 
total_df1.insert(5, '영업이익률', total_df1.pop('영업이익률'))
total_df1.drop(total_df1[total_df1['매출'] == 0].index, axis = 0, inplace = True)
total_df1.rename(columns={'낙찰금액(공기업)': '낙찰금액_공기업', '낙찰금액(국가기관)': '낙찰금액_국가기관', '낙찰금액(지자체)': '낙찰금액_지자체',
                         '입찰횟수(공기업)': '입찰횟수_공기업', '입찰횟수(국가기관)': '입찰횟수_국가기관', '입찰횟수(지자체)': '입찰횟수_지자체',
                         '평균참여수(공기업)': '평균참여수_공기업', '평균참여수(국가기관)': '평균참여수_국가기관', '평균참여수(지자체)': '평균참여수_지자체',
                         '낙찰수(공기업)': '낙찰수_공기업', '낙찰수(국가기관)': '낙찰수_국가기관', '낙찰수(지자체)': '낙찰수_지자체'}, inplace=True)
total_df1

,회사명,시기,매출,영업이익,총당기순이익,영업이익률,연구개발비,TobinsQ,업종분류코드,시장,...,입찰횟수_지자체,평균참여수_지자체,낙찰금액_지자체,낙찰수_지자체,입찰횟수_국가기관,평균참여수_국가기관,낙찰금액_국가기관,낙찰수_국가기관,낙찰여부,입찰여부
0,3S,20201,2.259202e+10,4.671810e+08,0.000000e+00,0.020679,5.323600e+07,2.191006,C,KOSDAQ,...,0.0,0.0,0.0,0.0,1.0,4.0,4.800000e+08,1.0,True,True
1,3S,20202,4.265512e+09,-3.499970e+08,-4.673000e+08,-0.082053,8.397100e+07,2.289685,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
2,3S,20203,1.097065e+10,-8.039280e+08,-1.181574e+09,-0.073280,9.066300e+07,2.044231,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
3,3S,20204,1.973514e+10,-6.666700e+08,-9.868740e+08,-0.033781,9.793400e+07,1.862129,C,KOSDAQ,...,0.0,0.0,0.0,0.0,8.0,4.0,2.221000e+09,5.0,True,True
4,3S,20211,2.347791e+10,-1.408266e+09,6.953350e+08,-0.059983,9.362800e+07,1.974030,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23435,힘스,20214,5.058420e+10,-4.371187e+09,-4.413920e+08,-0.086414,3.632138e+09,0.967528,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
23436,힘스,20221,9.017422e+09,-2.198637e+09,-1.605159e+09,-0.243821,9.508470e+08,1.062566,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
23437,힘스,20222,1.822716e+10,-6.597300e+09,-4.122215e+09,-0.361949,1.963682e+09,0.880589,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False
23438,힘스,20223,2.393546e+10,-1.253413e+10,-6.750329e+09,-0.523663,3.027635e+09,0.820003,C,KOSDAQ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,False,False


In [49]:
total_df1.describe()

,시기,매출,영업이익,총당기순이익,영업이익률,연구개발비,TobinsQ,종업원수,낙찰금액,입찰횟수_공기업,...,낙찰금액_공기업,낙찰수_공기업,입찰횟수_지자체,평균참여수_지자체,낙찰금액_지자체,낙찰수_지자체,입찰횟수_국가기관,평균참여수_국가기관,낙찰금액_국가기관,낙찰수_국가기관
count,23440.000000,2.344000e+04,2.344000e+04,2.344000e+04,23440.000000,2.344000e+04,23440.000000,2.344000e+04,2.344000e+04,23440.000000,...,2.344000e+04,23440.000000,23440.000000,23440.000000,2.344000e+04,23440.000000,23440.000000,23440.000000,2.344000e+04,23440.000000
mean,20212.793089,6.917971e+11,4.516792e+10,3.585275e+10,-1.227760,1.503053e+10,1.216811,4.919991e+04,1.780771e+08,0.111049,...,4.628468e+07,0.009727,0.101152,0.239610,2.545198e+07,0.015316,0.340145,0.314809,1.063404e+08,0.074445
std,8.169292,5.322658e+12,7.547494e+11,6.810462e+11,36.503202,3.467024e+11,1.790592,5.364994e+06,4.076281e+09,1.107359,...,3.033430e+09,0.127291,1.036004,14.021066,9.370161e+08,0.182598,5.171107,11.646832,2.452265e+09,0.748367
min,20201.000000,3.667000e+06,-3.265515e+13,-2.442911e+13,-3746.544324,0.000000e+00,0.000244,1.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,20204.000000,2.285277e+10,-9.265590e+08,-1.237357e+09,-0.028727,0.000000e+00,0.422355,8.900000e+01,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
50%,20213.000000,6.629937e+10,2.414543e+09,1.711654e+09,0.035295,4.265880e+08,0.754423,1.890000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
75%,20222.000000,2.220411e+11,1.291267e+10,1.059608e+10,0.092024,2.184132e+09,1.362350,4.290000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
max,20224.000000,3.022314e+14,5.163386e+13,5.565408e+13,0.832342,2.500422e+13,42.231655,6.952446e+08,3.643000e+11,49.000000,...,3.643000e+11,5.000000,47.000000,1509.000000,9.770000e+10,12.000000,456.000000,1154.000000,1.654788e+11,61.000000


In [50]:
df1 = total_df1[total_df1['영업이익률'] <= -5][['회사명', '업종분류코드']] # 대부분 바이오 기업
df1['회사명'].unique()
## 조달여부와 바이오 회사의 관계 
## 조달의 횟수와 바이오 기업의 가치 상승이 관련이 있을까? -> 어느 정도는 영향을 주겠지만 큰 의미를 주지 못한다고 판단 -> 이상치 삭제 

array(['CJ 바이오사이언스', 'CNT85', 'GKL', 'SK바이오팜', '고바이오랩', '나라소프트', '노랑풍선',
       '노브메타파마', '노을', '데이드림엔터', '딥노이드', '로보쓰리에이아이앤로보틱스', '로지스몬', '무진메디',
       '보로노이', '뷰노', '브릿지바이오테라퓨틱스', '샤페론', '서남', '셀리드', '셀리버리', '신라젠',
       '신테카바이오', '싸이토젠', '썸에이지', '아스타', '아이진', '아이톡시', '아퓨어스', '안지오랩',
       '압타머사이언스', '압타바이오', '애니메디솔루션', '에스씨엠생명과학', '에스엘테라퓨틱스', '에이비엘바이오',
       '에이비온', '에이프릴바이오', '엔솔바이오사이언스', '엘아이에스', '오스코텍', '올리패스', '올릭스',
       '유엑스엔', '유틸렉스', '이노테라피', '이오플로우', '제이엘케이', '젠큐릭스', '지나인제약',
       '지노믹트리', '지놈앤컴퍼니', '차백신연구소', '카이노스메드', '퀀타매트릭스', '큐러블', '큐리언트',
       '테크트랜스', '툴젠', '티움바이오', '파멥신', '펨토바이오메드', '펩트론', '프레스티지바이오로직스',
       '피플바이오', '하나투어', '헬릭스미스', '휴맥스홀딩스'], dtype=object)

In [51]:
def get_outlier(df = None, col = None): 
    out = df[col]
    quantile_25 = np.percentile(out.values, 25)
    quantile_75 = np.percentile(out.values, 75) 
    iqr = quantile_75 - quantile_25
    low_val = quantile_25 - (1.5 * iqr)
    high_val = quantile_75 + (1.5 * iqr)
    out_index = out[(out < low_val) | (out > high_val)].index
    return out_index

out_indx1 = get_outlier(total_df1, '영업이익률')
total_df1.drop(out_indx1, axis = 0, inplace = True) # 영업이익률 이상치 약 3000개 삭제
total_df1.reset_index(drop = True, inplace = True)

total_df1.describe()

,시기,매출,영업이익,총당기순이익,영업이익률,연구개발비,TobinsQ,종업원수,낙찰금액,입찰횟수_공기업,...,낙찰금액_공기업,낙찰수_공기업,입찰횟수_지자체,평균참여수_지자체,낙찰금액_지자체,낙찰수_지자체,입찰횟수_국가기관,평균참여수_국가기관,낙찰금액_국가기관,낙찰수_국가기관
count,20134.000000,2.013400e+04,2.013400e+04,2.013400e+04,20134.000000,2.013400e+04,20134.000000,2.013400e+04,2.013400e+04,20134.000000,...,2.013400e+04,20134.000000,20134.000000,20134.000000,2.013400e+04,20134.000000,20134.000000,20134.000000,2.013400e+04,20134.000000
mean,20212.756283,7.790608e+11,5.340114e+10,4.305797e+10,0.043511,1.650407e+10,0.972133,9.693064e+02,2.004393e+08,0.121585,...,5.376318e+07,0.011076,0.111801,0.274509,2.443431e+07,0.016887,0.376180,0.351727,1.222418e+08,0.082845
std,8.175851,5.686002e+12,7.424481e+11,6.876366e+11,0.087007,3.723564e+11,1.170730,9.513228e+03,4.343089e+09,1.152512,...,3.272937e+09,0.136027,1.105909,15.123187,7.401522e+08,0.193299,5.496764,12.556230,2.644832e+09,0.803120
min,20201.000000,7.035430e+08,-5.846499e+12,-5.215581e+12,-0.209388,0.000000e+00,0.000244,1.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,20204.000000,3.143012e+10,8.938725e+07,0.000000e+00,0.002398,0.000000e+00,0.388978,1.030000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
50%,20213.000000,8.290120e+10,3.206938e+09,2.455074e+09,0.041915,3.945830e+08,0.677731,2.130000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
75%,20222.000000,2.641685e+11,1.448974e+10,1.172053e+10,0.092972,2.074700e+09,1.169020,4.710000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
max,20224.000000,3.022314e+14,5.163386e+13,5.565408e+13,0.272725,2.500422e+13,42.231655,1.045154e+06,3.643000e+11,49.000000,...,3.643000e+11,5.000000,47.000000,1509.000000,7.524770e+10,12.000000,456.000000,1154.000000,1.654788e+11,61.000000


In [52]:
total_df1['연구개발비'] = total_df1['연구개발비'].apply(lambda x: x/1e+8)
display(total_df1['연구개발비'].describe(), np.log1p(total_df1['연구개발비']).describe())
# 결과해석시 원래_값 = exp(변환된_값) - 1 으로 변환해석 해석

count     20134.000000
mean        165.040727
std        3723.564006
min           0.000000
25%           0.000000
50%           3.945830
75%          20.747003
max      250042.160000
Name: 연구개발비, dtype: float64

count    20134.000000
mean         1.852031
std          1.859313
min          0.000000
25%          0.000000
50%          1.598545
75%          3.079476
max         12.429389
Name: 연구개발비, dtype: float64

In [53]:
total_df1['낙찰금액'] = total_df1['낙찰금액'].apply(lambda x: x/1e+8)
display(total_df1['낙찰금액'].describe(), np.log1p(total_df1['낙찰금액']).describe())
# 결과해석시 원래_값 = exp(변환된_값) - 1 으로 변환해석 해석

count    20134.000000
mean         2.004393
std         43.430895
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       3643.000000
Name: 낙찰금액, dtype: float64

count    20134.000000
mean         0.101290
std          0.554626
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          8.200837
Name: 낙찰금액, dtype: float64

In [54]:
total_df1['매출'] = total_df1['매출'].apply(lambda x: x/1e+8)
display(total_df1['매출'].describe(), np.log1p(total_df1['매출']).describe())
# 결과해석시 원래_값 = exp(변환된_값) - 1 으로 변환해석 해석

count    2.013400e+04
mean     7.790608e+03
std      5.686002e+04
min      7.035430e+00
25%      3.143012e+02
50%      8.290120e+02
75%      2.641685e+03
max      3.022314e+06
Name: 매출, dtype: float64

count    20134.000000
mean         6.919079
std          1.674486
min          2.083861
25%          5.753528
50%          6.721440
75%          7.879551
max         14.921534
Name: 매출, dtype: float64

In [55]:
# 이상치 제거한 데이터 total_df1으로 저장
total_df1.to_parquet('total_df.pqt')
# total_df1.to_csv('total_df1.csv') pqt 파일 불러온 후 csv로 저장 가능

In [56]:
total_df1.groupby(['입찰여부']).agg({'TobinsQ':np.mean})

,TobinsQ
입찰여부,
False,0.983854
True,0.876975


In [44]:
0.876975 - 0.983854

-0.10687900000000006

In [45]:
total_df1.columns

Index(['회사명', '시기', '매출', '영업이익', '총당기순이익', '영업이익률', '연구개발비', 'TobinsQ',
       '업종분류코드', '시장', '종업원수', '낙찰금액', '입찰횟수_공기업', '평균참여수_공기업', '낙찰금액_공기업',
       '낙찰수_공기업', '입찰횟수_지자체', '평균참여수_지자체', '낙찰금액_지자체', '낙찰수_지자체', '입찰횟수_국가기관',
       '평균참여수_국가기관', '낙찰금액_국가기관', '낙찰수_국가기관', '낙찰여부', '입찰여부'],
      dtype='object')

In [16]:
## total_df 저장 최종 데이터 저장 
## 완!!